In [1]:
import pandas as pd
import joblib
from pathlib import Path
import sys; sys.path.append("../")
from src.run_regression import export_cox_result
from stargazer.stargazer import Stargazer, LineLocation

# for stargazer compatibility
from lifelines.fitters.cox_time_varying_fitter import CoxTimeVaryingFitter
from stargazer.starlib import _extract_feature, _merge_dicts
from stargazer.translators import register_class
from stargazer.label import Label
from math import sqrt


In [2]:
# Create compatibility between Cox model and stargazer


# For features that are simple attributes of "model", establish the
# mapping with internal name:
linearmodels_map_base = {
    # "p_values": "pvalues",
    "cov_values": "params_",
    "cov_std_err": "standard_errors_",
    # "r2": "rsquared",
    # "degree_freedom": "df_model",
    # "degree_freedom_resid": "df_resid",
    # "nobs": "_n_examples",
}

# Mapping for linearmodels key parameters
# between-, within- and overall-R² values extracted for potential future stats display in Stargazer
# Note: We here use corr_... attributes for the R² as this matches the results obtained in Stata
linear_model_map_panel = {
    "between_r2": "corr_squared_between",
    "within_r2": "corr_squared_within",
    "overall_r2": "corr_squared_overall",
}

# IV specific statistics map
linear_model_map_iv = dict()


def extract_model_data(model):
    data = {}
    linearmodels_map = _merge_dicts(linearmodels_map_base, linear_model_map_panel)
    data["ngroups"] = f"{model._n_unique:,.0f}"
    data['nobs'] = f"{model._n_examples:,.0f}"

    for key, val in linearmodels_map.items():
        data[key] = _extract_feature(model, val)

    # Common two both FE & IV models
    data["dependent_variable"] = model.event_col
    data["cov_names"] = model.params_.index.values
    data["conf_int_low_values"] = model.confidence_intervals_['95% lower-bound']
    data["conf_int_high_values"] = model.confidence_intervals_['95% upper-bound']
    # data["resid_std_err"] = sqrt(model.model_ss / model.df_resid)
    # data["f_statistic"] = model.f_statistic.stat
    data["p_values"] = model.summary['p']
    data["r2_adj"] = None
    data["pseudo_r2"] = None
    data['lr_ratio'] = model.log_likelihood_ratio_test()

    return data


classes = [
    (CoxTimeVaryingFitter, extract_model_data),
]

for klass, translator in classes:
    register_class(klass, translator)
    
Stargazer._auto_stats = [('n', 'nobs', 'Observations'),
                   ('ngroups', 'ngroups', 'N. of groups')]
    



In [3]:
pickle_path = Path("data/pickles/")

In [4]:
fa_21 = joblib.load(pickle_path / 'cox_fa21.pkl')
sp_22 = joblib.load(pickle_path / 'cox_sp22.pkl')
pooled = joblib.load(pickle_path / 'cox_pooled.pkl')

In [5]:
model_table = Stargazer([fa_21, sp_22, pooled])

model_table.custom_columns(['Fall 2021', 'Spring 2022', 'Pooled'])
model_table.show_model_numbers(False)
model_table.rename_covariates({
    'academic_career_GM' : 'GM',
    'academic_career_GR' : 'GR',
    'academic_career_LA' : 'LA',
    'academic_career_UG_A' : 'UG-A',
    'academic_career_UG_G' : 'UG-G',
    'academic_career_VM' : 'VM',
    'class_positivity' : 'Class Exposure',
    'current_gender_M' : 'Sex (M)',
})

model_table.covariate_order(['class_positivity', 'current_gender_M', 
                             'academic_career_UG_G', 'academic_career_UG_A',
                             'academic_career_GM', 'academic_career_GR',
                             'academic_career_LA', 'academic_career_VM'])

model_table.dep_var_name = None


In [17]:
print(model_table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & \multicolumn{1}{c}{Fall 2021} & \multicolumn{1}{c}{Spring 2022} & \multicolumn{1}{c}{Pooled}  \\
\hline \\[-1.8ex]
 Class Exposure & 1.855$^{***}$ & 0.266$^{}$ & 1.660$^{***}$ \\
& (0.199) & (0.894) & (0.193) \\
 Sex (M) & 0.063$^{}$ & -0.067$^{}$ & 0.038$^{}$ \\
& (0.082) & (0.145) & (0.071) \\
 UG-G & 1.480$^{***}$ & 0.876$^{***}$ & 1.337$^{***}$ \\
& (0.090) & (0.161) & (0.078) \\
 UG-A & 0.732$^{***}$ & -0.242$^{}$ & 0.545$^{***}$ \\
& (0.161) & (0.420) & (0.149) \\
 GM & 0.158$^{}$ & 0.313$^{}$ & 0.202$^{}$ \\
& (0.267) & (0.392) & (0.220) \\
 GR & -0.930$^{**}$ & -0.745$^{*}$ & -0.781$^{***}$ \\
& (0.413) & (0.421) & (0.294) \\
 LA & 0.570$^{***}$ & 0.080$^{}$ & 0.413$^{**}$ \\
& (0.203) & (0.367) & (0.178) \\
 VM & 0.250$^{}$ & -0.540$^{}$ & 0.061$^{}$ \\
& (0.309) & (0.589) & (0.274) \\
\hline \\[-1.8ex]
 Observations & 1,771,641 & 163,161 & 1,934,802 \\
 N.